In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# os.environ['CUDA_VISIBLE_DEVICES'] = ''

import tensorflow as tf

In [ ]:
from keras_vggface.vggface import VGGFace
model = VGGFace(model='vgg16')
model.summary()

In [ ]:
encoder = tf.keras.Model(inputs=model.input, outputs=model.get_layer('pool1').output)

In [ ]:
encoder.output.shape[1:]

In [ ]:

# build decoder
latent = tf.keras.Input(shape=encoder.output.shape[1:])

x = tf.keras.layers.UpSampling2D((2, 2))(latent)
x = tf.keras.layers.Conv2DTranspose(64, (3, 3), padding='same', activation='relu')(x)
x = tf.keras.layers.Conv2DTranspose(3, (3, 3), padding='same', activation='sigmoid')(x)

decoder = tf.keras.Model(inputs=latent, outputs=x)

decoder.summary()

In [ ]:
# set encoder as non-trainable
for layer in encoder.layers:
    layer.trainable = False

model = tf.keras.Sequential([encoder, decoder])

model.summary()

In [ ]:
# train
model.compile(optimizer='adam', loss='binary_crossentropy')